In [15]:
# imports
import numpy as np
import pandas as pd
import scipy as sp 
import math
from calc_mic_distances import calculate_distance

steps:
* import meaasured 3d mic positions and distance measurements
* only use points with measured distance
* calculate sum of squarred errors
* build a mic positions model with parameters (offsets and tilt)
* optimize those paramters for minimal sum of squarred errors
* return optimized paramters and mic positions

In [16]:
# read meaasured 3d mic positions

df_mic_coords = pd.read_excel("data/3d_measured_coords.xlsx", index_col=0)

_ = calculate_distance(df_mic_coords, "A1", "B2")

# read distance measurements
df_distances = pd.read_excel("data/mic_pos_distances.xlsx")

1.803 = distance A1 to B2


In [17]:
# check if calculated distances match

print(df_distances.head())

print("")
for index, row in df_distances.iterrows():
    
    dist = calculate_distance(df_mic_coords, row[0], row[1], print_dist=False)

    if math.isclose(dist, df_distances["dist calculated"][index], abs_tol=0.001) == False:
        print(index)
        print("different distance")

print("all distances checked")

  Mic_Index 1 Mic_Index 2  dist measured  dist calculated  error
0         D13         D16          1.260            1.256  0.004
1         B17         C16          0.670            0.673 -0.003
2         C10         D10          1.175            1.181 -0.006
3          C2         B15          1.650            1.651 -0.001
4          A7          B6          1.416            1.397  0.019

all distances checked


In [18]:
# get all mic positions used for measurng distances
# get indixes
measured_mic_pos = []

for mic_index_temp in df_distances["Mic_Index 1"]:
    measured_mic_pos.append(mic_index_temp)

for mic_index_temp in df_distances["Mic_Index 2"]:
    measured_mic_pos.append(mic_index_temp)

measured_mic_pos = set(measured_mic_pos) # remove double values
measured_mic_pos = sorted(measured_mic_pos) # sort values

# filter dataframe
df_mic_coords_filtered = df_mic_coords[df_mic_coords.isin(measured_mic_pos).any(axis=1)]


print(len(measured_mic_pos), "different mics used for measurng distances")
print("")
print(df_mic_coords_filtered)

25 different mics used for measurng distances

   Mic_Index  Plane      X      Y      Z
5         B6  right  1.370  0.989  0.629
7         B8  right  1.370  1.315  0.718
12       B13  right  1.370  0.157  1.257
14       B15  right  1.370  1.044  1.485
15       B16  right  1.370  1.392  1.510
16       B17  right  1.370  0.674  1.689
18        C2  front  0.913  0.028  0.267
20        C4  front  1.322  0.028  0.380
21        C5  front  0.571  0.028  0.425
26       C10  front  0.299  0.028  0.990
32       C16  front  1.313  0.028  1.508
34        D1   left  0.028  0.817  0.023
36        D3   left  0.028  1.215  0.289
38        D5   left  0.028  0.905  0.427
40        D7   left  0.028  1.331  0.677
43       D10   left  0.028  1.177  0.990
46       D13   left  0.028  1.392  1.260
49       D16   left  0.028  0.161  1.511
50       D17   left  0.028  0.877  1.690
51        A1    top  0.590  1.246  1.871
54        A4    top  1.376  0.864  1.871
55        A5    top  1.028  0.857  1.871
56        

In [19]:
# calculate sum of squarred errors

for index, row in df_distances.iterrows():
    
    dist = calculate_distance(df_mic_coords_filtered, row[0], row[1], print_dist=False)
    sq_error = (dist - df_distances["dist measured"][index])**2
    df_distances.loc[index, "squared error"] = sq_error

sum_sq_errors_default = df_distances["squared error"].sum()
print(round(sum_sq_errors_default, 5), "= default sum of squared errors")


0.00696 = default sum of squared errors


In [20]:
# TODO# build parametric model

# parameter:

# translations = shift = offset

# Rotations:
# Roll (rotation around the plane's local X-axis)
# Pitch (rotation around the plane's local Y-axis)
# Yaw (rotation around the plane's local Z-axis) --> assumption: no yaw

# shift is in m
# roll & pitch in degrees

# top plane
A_x_shift = 0.2
A_y_shift = 0

# left plane
# B_x_shift = 0 # keep 0 for reference
B_y_shift = 0
B_z_shift = 0
B_roll = 0 # rotation over 3d y axis 
B_pitch = 0 # rotation over 3d z axis

# front plane
C_x_shift = 0
# C_y_shift = 0 # keep 0 for reference
C_z_shift = 0
C_roll = 0 # rotation over 3d x axis 
C_pitch = 0 # rotation over 3d z axis

# right_plane
D_x_shift = 0
D_y_shift = 0
D_z_shift = 0
D_roll = 0 # rotation over 3d y axis 
D_pitch = 0 # rotation over 3d z axis


initial_parameters = [
    A_x_shift,
    A_y_shift,
    B_y_shift,
    B_z_shift,
    B_roll,
    B_pitch,
    C_x_shift,
    C_z_shift,
    C_roll,
    C_pitch,
    D_x_shift,
    D_y_shift,
    D_z_shift,
    D_roll,
    D_pitch
    ]


print(len(initial_parameters), "parameter in total")
print(initial_parameters)

15 parameter in total
[0.2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [21]:
# parametric model

def pandas_rotation(df_mic_coords, plane_to_rotate, roll, pitch, roll_axis, pitch_axis):
    
    # create rotation object
    planes_to_rotate_formatted = roll_axis+pitch_axis
    r = sp.spatial.transform.Rotation.from_euler(planes_to_rotate_formatted, [roll, pitch], degrees=True)
    
    # filter plane to rotate (left, rigth, etc.)
    filtered_df = df_mic_coords[df_mic_coords['Plane'].isin([plane_to_rotate])]

    coordinates = filtered_df[["X_optimized", "Y_optimized", "Z_optimized"]].to_numpy()

    # Apply the rotation to the filtered subset of coordinates
    rotated_coordinates = r.apply(coordinates)

    # Update the original DataFrame with the rotated coordinates
    df_mic_coords.loc[filtered_df.index, ['X_optimized', 'Y_optimized', 'Z_optimized']] = rotated_coordinates

    return df_mic_coords



def parametric_model(df_mic_coords, parameters):

    # copy initial coords to preserve them
    df_mic_coords["X_optimized"] = df_mic_coords["X"]
    df_mic_coords["Y_optimized"] = df_mic_coords["Y"]
    df_mic_coords["Z_optimized"] = df_mic_coords["Z"]

    # top plane x,y shift
    df_mic_coords.loc[df_mic_coords["Plane"].isin(["top"]), "X_optimized"] += parameters[0]
    df_mic_coords.loc[df_mic_coords["Plane"].isin(["top"]), "Y_optimized"] += parameters[1]

    # left plane y,z shift and rotation
    df_mic_coords.loc[df_mic_coords["Plane"].isin(["left"]), "Y_optimized"] += parameters[2]
    df_mic_coords.loc[df_mic_coords["Plane"].isin(["left"]), "Z_optimized"] += parameters[3]
    df_mic_coords = pandas_rotation(df_mic_coords, "left", parameters[4], parameters[5], "y", "z")

    # front plane x,z shift and rotation
    df_mic_coords.loc[df_mic_coords["Plane"].isin(["front"]), "X_optimized"] += parameters[6]
    df_mic_coords.loc[df_mic_coords["Plane"].isin(["front"]), "Z_optimized"] += parameters[7]
    df_mic_coords = pandas_rotation(df_mic_coords, "front", parameters[8], parameters[9], "x", "z")

    # right plane x,y,z shift & rotatiotn
    df_mic_coords.loc[df_mic_coords["Plane"].isin(["right"]), "X_optimized"] += parameters[10]
    df_mic_coords.loc[df_mic_coords["Plane"].isin(["right"]), "Y_optimized"] += parameters[11]
    df_mic_coords.loc[df_mic_coords["Plane"].isin(["right"]), "Z_optimized"] += parameters[12]
    df_mic_coords = pandas_rotation(df_mic_coords, "right", parameters[13], parameters[14], "y", "z")

    return df_mic_coords

In [22]:
def calc_error_optimized_coords(df_distances, df_mic_coords_moved):

    for index, row in df_distances.iterrows():
        dist_optimized = calculate_distance(
            df_mic_coords_moved, row[0], row[1], print_dist=False, 
            x_col="X_optimized", y_col="Y_optimized", z_col="Z_optimized"
            )    
        sq_error = (dist_optimized - df_distances["dist measured"][index])**2
        df_distances.loc[index, "squared error optimized"] = sq_error

    sum_sq_errors = df_distances["squared error optimized"].sum()
    return sum_sq_errors, df_distances


In [23]:

def optimizer_wrapper(initial_parameters, df_mic_coords=df_mic_coords, df_distances=df_distances):
    df_mic_coords_moved = parametric_model(df_mic_coords, initial_parameters)
    sum_sq_errors, _ = calc_error_optimized_coords(df_distances, df_mic_coords_moved)

    return sum_sq_errors

In [24]:
result = sp.optimize.minimize(optimizer_wrapper, initial_parameters)

print("Optimal variables:", result.x)
print("Optimal function value:", result.fun)

Optimal variables: [-0.0380209  -0.03926696 -0.03352994 -0.02157031 -0.16013974 -0.38830788
  0.08006996 -0.00557273  1.06346201  1.60886169  0.0404717   0.01607017
  0.01144947 -0.71111458 -0.67249499]
Optimal function value: 1.3478284241642126e-05


In [25]:
import plotly.express as px

df_mic_coords_optimized = parametric_model(df_mic_coords, result.x)

# Create the first 3D scatter plot
fig = px.scatter_3d(df_mic_coords, 
                    x='X', 
                    y='Y', 
                    z='Z', 
                    text='Mic_Index', 
                    title='3D Scatter Plot', 
                    color='Plane', 
                    width=1000, 
                    height=1000)

# Create the second 3D scatter plot (optimized data)
fig_optimized = px.scatter_3d(df_mic_coords_optimized, 
                               x='X_optimized', 
                               y='Y_optimized', 
                               z='Z_optimized', 
                               text='Mic_Index', 
                               color='Plane')

# Customize the color of the second plot to differentiate it
for trace in fig_optimized.data:
    trace.marker.color = 'rgba(0, 0, 0, 0.7)'  # black with some transparency
    trace.name = 'Optimized Data'  # Label in the legend

# Add each trace from the second figure to the first
for trace in fig_optimized.data:
    fig.add_trace(trace)

# Display the combined figure
fig.show()
